## Explore course features

In [1]:
#import modules
import pandas as pd
import numpy as np

import requests

from pandas.io import gbq
from google.oauth2 import service_account

import gspread
import json, os
from canvasapi import Canvas

from datetime import datetime

from time import time

from params import basedir, run_mode

In [2]:
pd.set_option("display.max_columns", 200)

In [3]:
print(run_mode)

dev


In [4]:
if run_mode == 'dev':
    out_table = "all_courses4"
    setting_out_table = "all_course_settings4"
    feature_flags_out_table = "feature_flags_table4"
elif run_mode == 'prod':
    out_table = "all_courses4"
    setting_out_table = "all_course_settings4"    
    feature_flags_out_table = "feature_flags_table4"

elif run_mode == 'prod_home':
    out_table = "all_courses4"
    setting_out_table = "all_course_settings4"    
    feature_flags_out_table = "feature_flags_table4"    

    
elif run_mode == 'mig':
    out_table = "all_courses4"
    setting_out_table = "all_course_settings4"    
    feature_flags_out_table = "feature_flags_table4"


In [5]:
#get Google cloud credentials
project_id = 'canvas-portal-data-custom'
cred_file = '{}/canvas-portal-data-custom-6e244db3b826.json'.format(basedir)
data_dl = 'data'
scopes = [ "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
            "https://spreadsheets.google.com/auth/spreadsheets"]
credentials = service_account.Credentials.from_service_account_file(cred_file,)


from instances import canvas_api, panopto_api

canvas_API_KEY = canvas_api['ACCES_TOKEN']
canvas_API_URL = canvas_api['API_URL']#+'/accounts/1'


In [6]:
# Initialize a new Canvas object
canvas = Canvas(canvas_API_URL, canvas_API_KEY)
canvas.__dict__


{'_Canvas__requester': <canvasapi.requester.Requester at 0x7f9ea0cb5070>}

In [7]:
#Get a list of courses from GBQ:

#Send out course tabs dataframe to GBQ
query_ = '''SELECT `course_id`, `course_sis_id`, `Dept`,`enrollment_term`,
       `parent_account`, `Course_name`, `course_state` FROM `lt_courses.{}`
'''.format(out_table)
all_courses_df = pd.read_gbq(query_, project_id, credentials=credentials)

In [8]:
def get_course_settings_new_analytics_info(course_id):
    feature_flags = None
    all_settings = None
    try:
        c1 = canvas.get_course(course_id)
    except:
        c1 = None
    try:    
        #Get feature flags for new analytics for course
        all_features = c1.get_features()
        for feature_ in all_features:
            if feature_.feature == "analytics_2":
                feature_flags = feature_.feature_flag
                feature_flags['course_id'] = course_id
                
    except Exception as e:
        pass                

    try:    
        #Get all settings for this course
        all_settings = c1.get_settings()
        all_settings['course_id'] = course_id

    except Exception as e:
        pass
    
    return (feature_flags, all_settings)


In [9]:
#get the course tab info for a set number of courses at a time

all_course_list_0 = all_courses_df.course_id.tolist()

if run_mode == 'dev' or run_mode=='mig':
    course_pos_list = all_course_list_0[:5]
elif run_mode == 'prod' or run_mode=='prod_home':
    course_pos_list = all_course_list_0
    
#all_course_list_0 = [3156, 13894]
all_feature_flags_list = []
all_settings_list = []
for cid in course_pos_list:
    #print(cid)
    feature_flags, settings = get_course_settings_new_analytics_info(cid)
    if feature_flags != None:
        all_feature_flags_list.append(feature_flags)
    if settings != None:
        all_settings_list.append(settings)
    
new_analytics_df = pd.DataFrame(all_feature_flags_list)
course_settings_df = pd.DataFrame(all_settings_list)

In [10]:
course_settings_df = course_settings_df.merge(all_courses_df, on='course_id', how='left')
new_analytics_df = new_analytics_df.merge(all_courses_df, on='course_id', how='left')

#save local copies:
course_settings_df.to_csv('{}.csv'.format(setting_out_table), index=None)
new_analytics_df.to_csv('{}.csv'.format(feature_flags_out_table), index=None)

#Export to GBQ:
course_settings_df.to_gbq('lt_courses.{}'.format(setting_out_table), project_id, if_exists='replace', 
                          credentials=credentials)
new_analytics_df.to_gbq('lt_courses.{}'.format(feature_flags_out_table), project_id, if_exists='replace', 
                          credentials=credentials)

1it [00:04,  4.12s/it]
1it [00:07,  7.64s/it]
